In [1]:
import numpy as np
from ISLP import load_data
import statsmodels.api as sm

from ISLP.models import (ModelSpec as MS, summarize, poly)
from sklearn.model_selection import train_test_split

In [2]:
from functools import partial
from sklearn.model_selection import (cross_validate, KFold, ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

In [3]:
import warnings
warnings.filterwarnings('ignore')

### validation set approach

In [27]:
Auto = load_data('Auto')
print(Auto.shape)
# As there are 392 observations, we split into two equal sets of size 196
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=0)

(392, 8)


In [5]:
X_train = Auto_train['horsepower'].values
X_train = sm.add_constant(X_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()
results.params

const    39.905465
x1       -0.156307
dtype: float64

In [6]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.608
Model:                            OLS   Adj. R-squared:                  0.606
Method:                 Least Squares   F-statistic:                     300.4
Date:                Sun, 16 Feb 2025   Prob (F-statistic):           2.83e-41
Time:                        22:48:51   Log-Likelihood:                -590.83
No. Observations:                 196   AIC:                             1186.
Df Residuals:                     194   BIC:                             1192.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         39.9055      1.009     39.537      0.000      37.915      41.896
x1            -0.1563      0.009    -17.333      0.000      -0.174      -0.139
==============================================================================
Omnibus:                        7.263   Durbin-Watson:                   2.175
Prob(Omnibus):                  0.026   Jarque-Bera (JB):                6.993
Skew:                           0.440   Prob(JB):                       0.0303
Kurtosis:                       3.286   Cond. No.                         319.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
X_valid = Auto_valid['horsepower'].values
X_valid = sm.add_constant(X_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred)**2)

23.61661706966988

In [8]:
# evalMSE() that takes a model string as well as a training and test set and returns the MSE on the test set
def evalMSE(train, test, d):
    X_train = train['horsepower']
    X_train = np.power.outer(X_train.values, np.arange(d+1))
    y_train = train['mpg']
    
    X_test = test['horsepower']
    X_test = np.power.outer(X_test.values, np.arange(d+1))
    y_test = test['mpg']
    
    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)
    return np.mean((y_test - test_pred)**2)

In [9]:
MSE = np.zeros(3)

for idx, degree in enumerate(range(1, 4)):
    print(idx, degree)
    MSE[idx] = evalMSE(Auto_train, Auto_valid, degree) 

MSE

0 1
1 2
2 3


array([23.61661707, 18.76303135, 18.79694163])

try a different training, validation split by setting a different random_state

the mses are different

In [10]:
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=3)

MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE(Auto_train, Auto_valid, degree) 
    
MSE

array([20.75540796, 16.94510676, 16.97437833])

an **alternative** implementation of polynomial features

use PolynomialFeatures

In [11]:
from sklearn.preprocessing import PolynomialFeatures

In [12]:
Auto_train[['horsepower']].shape

(196, 1)

In [13]:
# evalMSE() that takes a model string as well as a training and test set and returns the MSE on the test set
def evalMSE(train, test, d):
    poly_features = PolynomialFeatures(degree=d)
    X_train = poly_features.fit_transform(train[['horsepower']]) 
    y_train = train['mpg']
    
    X_test = poly_features.fit_transform(test[['horsepower']])
    y_test = test['mpg']
    
    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)
    return np.mean((y_test - test_pred)**2)

In [14]:
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=3)
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE(Auto_train, Auto_valid, degree) 
    
MSE

array([20.75540796, 16.94510676, 16.97437833])

### cross validation

LOOCV

In [15]:
M = sklearn_sm(sm.OLS)
H = np.array(Auto['horsepower'])
X = sm.add_constant(H)
Y = Auto['mpg']
cv_results = cross_validate(M, X, Y, cv=Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err

24.231513517929226

an alternative implementation of LOOCV without adding a constant term

In [16]:
# an intercept term is included
hp_model = sklearn_sm(sm.OLS, MS(['horsepower']))
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
## cv = Auto.shape[0] is LOOCV
cv_results = cross_validate(hp_model, X, Y, cv=Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err

24.231513517929226

LOOCV for polynomial regressions

define the polynomials of horsepower using np.power.outer

In [17]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
# if terms are not specified in sklean_sm, then sm.OLS uses all the columns in X without including an intercept term
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    print(X.shape, d)
    M_CV = cross_validate(M, X, Y, cv=Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

(392, 2) 1
(392, 3) 2
(392, 4) 3
(392, 5) 4
(392, 6) 5


array([24.23151352, 19.24821312, 19.33498406, 19.42443032, 19.03322078])

K-fold cross validation

In [18]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])

cv = KFold(n_splits=10, shuffle=True, random_state=0) # use same splits for each degree

for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M, X, Y, cv=cv)
    cv_error[i] = np.mean(M_CV['test_score'])
    
cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.478484  , 19.1372021 ])

### bootstrap

Estimating the Accuracy of a Statistic of Interest

In [19]:
Portfolio = load_data('Portfolio')

def alpha_func(D, idx):
    cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
    return ((cov_[1,1] - cov_[0,1]) / (cov_[0,0]+cov_[1,1]-2*cov_[0,1]))

alpha_func(Portfolio, range(100))

0.57583207459283

In [20]:
rng = np.random.default_rng(0)
alpha_func(Portfolio, rng.choice(100, 100, replace=True))

0.6074452469619002

In [21]:
def boot_SE(func, D, n=None, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index, n, replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B)**2)

In [22]:
alpha_SE = boot_SE(alpha_func, Portfolio, B=1000, seed=0)
alpha_SE

0.09118176521277607

Estimating the Accuracy of a Linear Regression Model

In [35]:
def boot_OLS(X, Y, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    n = X.shape[0]
    first_, second_ = np.zeros((2,)), np.zeros((2,))
    
    for _ in range(B):
        idx = rng.choice(n, n, replace=True)
        Y_ = Y.iloc[idx]
        X_ = X.iloc[idx]
        # add constant
        X_ = sm.add_constant(X_)
        # sm.OLS does not automatically include an intercept term
        value = sm.OLS(Y_, X_).fit().params.values
        first_ += value
        second_ += value * value
    
    return np.sqrt(second_ / B - (first_ / B)**2)

In [36]:
boot_OLS(Auto['horsepower'], Auto['mpg'])

array([0.85785417, 0.00745836])

polynomial regression

In [37]:
def boot_polynomial_regression(X, Y, d, B=1000, seed=0):
    # polynomial degree is d
    rng = np.random.default_rng(seed)
    n = X.shape[0]
    first_, second_ = np.zeros((d+1,)), np.zeros((d+1,))
    
    org_Y = Y.copy()
    org_X = X.copy()
    org_X = np.power.outer(org_X.values, np.arange(d+1))
    print(sm.OLS(org_Y, org_X).fit().params)
    
    for _ in range(B):
        idx = rng.choice(n, n, replace=True)
        Y_ = Y.iloc[idx]
        X_ = X.iloc[idx]
        # construct polynomials of X
        X_ = np.power.outer(X_.values, np.arange(d+1))
        value = sm.OLS(Y_, X_).fit().params.values
        first_ += value
        second_ += value * value
        
        
    return np.sqrt(second_ / B - (first_ / B)**2)

In [38]:
# polynomial degree is 1
boot_polynomial_regression(Auto['horsepower'], Auto['mpg'], 1)

const    39.935861
x1       -0.157845
dtype: float64


array([0.85785417, 0.00745836])

In [39]:
boot_polynomial_regression(Auto['horsepower'], Auto['mpg'], 2)

const    56.900100
x1       -0.466190
x2        0.001231
dtype: float64


array([2.06783958e+00, 3.30188774e-02, 1.19712226e-04])

**Exercise**: Use bootstrap to compute the mean of mpg

In [28]:
def boot_mean(X, B=1000, seed=0):
    pass

In [29]:
boot_mean(Auto['mpg'])

In [30]:
boot_mean(Auto['horsepower'])